In [1]:
import pandas as pd 
import requests
import json


In [2]:
def getValues(tagList, start ,end):
        url = "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
        d = {
            "metrics": [
                {
                    "tags": {},
                    "name": ""
                }
            ],
            "plugins": [],
            "cache_time": 0,
            "cache_time": 0,
            "start_absolute": start,
            "end_absolute": end
        }
        finalDF = pd.DataFrame()
        for tag in tagList:
            d['metrics'][0]['name'] = tag
            res = requests.post(url=url, json=d)
            values = json.loads(res.content)
            df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
            finalDF = pd.concat([finalDF, df], axis=1)

        finalDF = finalDF.loc[:, ~finalDF.columns.duplicated()]
        finalDF.dropna(subset=['time'], inplace=True)
        # finalDF['time'] = pd.to_datetime(finalDF['time'], unit='ms').dt.strftime('%d-%m-%y %H:%M:%S')
        return finalDF

In [3]:
def mld_2_anode_status(start, end):
    tags= [
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric',
            'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'
        ]
    
    temp_df = pd.DataFrame(columns=['time'])
    temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
    temp_df.loc[1, 'time'] = end
    temp_df['time'] += (5 * 60 + 30) * 60 * 1000
    temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

    mld_2_df=getValues(tags, start, end)
    
    mld_2_df = mld_2_df.dropna()
    mld_2_df = mld_2_df[(mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.56) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] <= 1.69)]
    
    mld_2_df = mld_2_df[mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] % 1 == 0]
    mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].astype(int)

    # Find consecutive duplicate values in the column and mark them for removal
    mld_2_df['to_remove'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] == mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].shift(1)

    mld_2_df = mld_2_df[mld_2_df['to_remove'] != True]
    
    mld_2_df.reset_index(drop=True, inplace=True)
    # print(mld_2_df)
    # Assuming df is your DataFrame
    # Convert the 'time' column to datetime format
    mld_2_df['time'] += (5 * 60 + 30) * 60 * 1000
    mld_2_df['date'] = pd.to_datetime(mld_2_df['time'], unit='ms')

    # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
    condition_better = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.66
    condition_good = (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.65) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.66)
    condition_bad = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.65

    # Create a new column 'category' based on conditions
    mld_2_df['category'] = 'Undefined'
    mld_2_df.loc[condition_better, 'category'] = 'Better'
    mld_2_df.loc[condition_good, 'category'] = 'Good'
    mld_2_df.loc[condition_bad, 'category'] = 'Bad'

    # Create an empty DataFrame with columns
    hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

    # Group by hour and category, then count occurrences
    hourly_counts = mld_2_df.groupby([mld_2_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

    complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
    reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

    # Add columns for 'Good', 'Better', and 'Bad' with zeros
    reference_df['Good'] = 0
    reference_df['Better'] = 0
    reference_df['Bad'] = 0

    missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
    for category in missing_categories:
            hourly_counts[category] = 0

    hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
    hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
    hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

    hourly_counts = hourly_counts.fillna(0)
    return hourly_counts

In [4]:
start = 1701369000000  # define start time epoch
end = 1703961000000   # end time epoch

In [5]:
hourly_counts_mld_2 = mld_2_anode_status(start, end)

In [6]:
total_anode_mld2 = hourly_counts_mld_2['Bad'].sum() + hourly_counts_mld_2['Better'].sum() + hourly_counts_mld_2['Good'].sum()
print(total_anode_mld2)

5458.0


In [7]:
def mld_1_anode_status(start, end):
        tags= [
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
                'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'
            ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
            & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')

        # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
        condition_better = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.66
        condition_good = (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.65) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.66)
        condition_bad = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.65

        # Create a new column 'category' based on conditions
        mld_1_df['category'] = 'Undefined'
        mld_1_df.loc[condition_better, 'category'] = 'Better'
        mld_1_df.loc[condition_good, 'category'] = 'Good'
        mld_1_df.loc[condition_bad, 'category'] = 'Bad'

        # Create an empty DataFrame with columns
        hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

        # Group by hour and category, then count occurrences
        hourly_counts = mld_1_df.groupby([mld_1_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        # Add columns for 'Good', 'Better', and 'Bad' with zeros
        reference_df['Good'] = 0
        reference_df['Better'] = 0
        reference_df['Bad'] = 0

        missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
        for category in missing_categories:
                hourly_counts[category] = 0

        hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
        hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
        hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

        hourly_counts = hourly_counts.fillna(0)
        return hourly_counts

In [8]:
hourly_counts_mld_1 = mld_1_anode_status(start, end)

In [9]:
total_anode_mld1 = hourly_counts_mld_1['Bad'].sum() + hourly_counts_mld_1['Better'].sum() + hourly_counts_mld_1['Good'].sum()
print(total_anode_mld1)

5466.0


In [10]:
total_anode = total_anode_mld1 + total_anode_mld2
print(total_anode)

10924.0
